In [29]:
!pip install transformers
!pip install gTTS

In [44]:
from transformers import VisionEncoderDecoderModel,ViTFeatureExtractor,AutoTokenizer
import torch
from PIL import Image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
model=VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor=ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer=AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
device=torch.device("cuda"if torch.cuda.is_available()else"cpu")
model.to(device)
max_length=16
num_beams=4
gen_kwargs={"max_length":max_length,"num_beams":num_beams}
from PIL import Image
from gtts import gTTS
import os
def predict_caption(image_paths):
    images = []
    for image_path in image_paths:
        i_image = Image.open(image_path)
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode='RGB')
        images.append(i_image)
    pixel_values = feature_extractor(images=images, return_tensors='pt').pixel_values
    pixel_values = pixel_values.to(device)
    output_ids = model.generate(pixel_values, **gen_kwargs)
    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    print("Final captions:", preds)
    for caption in preds:
        tts = gTTS(caption)
        tts.save("temp_audio.mp3")
        os.system("start temp_audio.mp3")

    return preds
predict_caption(["/content/mountain.jpg"])

Final captions: ['a grassy area with trees and a road']


['a grassy area with trees and a road']